# W.3. Bond Futures

## FINM 37500 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

#### Notation Commands

$$\newcommand{\cpn}{\text{c}}$$
$$\newcommand{\Tunder}{T_\text{bond}}$$
$$\newcommand{\accint}{\xi}$$
$$\newcommand{\convert}{\psi}$$
$$\newcommand{\ctd}{\text{ctd}}$$

$$\newcommand{\refrate}{r^{\text{ref}}}$$
$$\newcommand{\pfwd}{P_{\text{fwd}}}$$
$$\newcommand{\grossbasis}{\Gamma}$$
$$\newcommand{\netbasis}{\tilde{\grossbasis}}$$
$$\newcommand{\pnl}{\Pi}$$

$$\newcommand{\years}{\tau}$$
$$\newcommand{\yearsACT}{\years_{\text{act/360}}}$$

$$\newcommand{\fwdprice}{\mathcal{F}_{t\to T}}$$
$$\newcommand{\futprice}{F_{t\to T}}$$

$$\newcommand{\fwdpriceT}{\mathcal{F}_{T\to T}}$$
$$\newcommand{\futpriceT}{F_{T\to T}}$$

$$\newcommand{\fwdpriceDT}{\mathcal{F}_{t+h\to T}}$$
$$\newcommand{\futpriceDT}{F_{t+h\to T}}$$

$$\newcommand{\rrepo}{r^{\text{repo}}}$$
$$\newcommand{\carry}{\widetilde{\cpn}}$$

# Forward Pricing

### Basics
A **forward** agreement is a commitment at $t$ to purchase an asset at time $T$ for a price $\fwdprice$.
* The rate is set such that the **value** of the forward is \$0 when it is entered.
* The value will change over time.
* Unlike an option, the net value at expiration could be to either party.

This is similar to
* swaps
* futures

Note then that $\fwdprice(t,T)$ is not the **value** of the futures contract. 
* Rather, at the time the contract is entered, the forward price is set such that the value of the contract is 0.

### Synthetic Forwards

Replicate the forward by 
* purchasing the asset at $t$
* immediately repo it for term to $T$
* at $T$, close the repo by repurchasing at $P_t (1+\rrepo\yearsACT)$ where 
    * $\rrepo$ denotes the simply compounded repo rate
    * $\yearsACT$ denotes the forward interval, $T-t$ measured in (ACT/360)

### Carry and the Forward Drop?

Do you think the forward price is higher or lower than the spot?

The **forward drop** is commonly seen as the futures price is typically lower than the spot price.
* If coupon is higher than repo, the spot is more attractive, all else equal. Accordingly, future price must be lower than spot.
* If coupon is lower than repo, than the forward price should be higher.

### Carry
Recall from FINM 36700 or FINM 37400 that **carry** (in this context) refers to the difference of the coupon of bond $i$ over the short-term (repo) interest rate.

$$\carry^i = \cpn^i-\rrepo$$

The logic of the forward drop (or forward rise) can be stated as
$$\years\carry^i = P_t^i - \fwdprice^i$$

#### Note
This relationship is analogous to Covered Interest Parity in FX (See FINM 36700) and will approximately hold by no-arbitrage.

## Example

T-note with the following specs:
* coupon is 4\%
* matures 2030-02-28
* issued 2023-02-28
* first coupon 2023-08-31

Now consider pricing a forward in the following environment...
* $t=$2023-04-18
* forward maturity $T=$2023-08-01
* spot price $P_t = 102-02$
* repo rate is $.0485$

<img src="../refs/Bloomberg synthetic forward answer.png" />

Reference: Bloomberg

In [1]:
from datetime import datetime
from datetime import date

date_issue = date(2023,2,28)
date_coupon = date(2023,8,31)
date_now = date(2023,4,18)
date_fwd = date(2023,8,1)

days_cpn = (date_coupon - date_issue).days
days_accint0 = (date_now - date_issue).days
days_accint1 = (date_fwd - date_issue).days
days_fwd = (date_fwd - date_now).days

In [2]:
import pandas as pd
import numpy as np

spot = 102+2/32
repo = .0485
cpn = .04
face = 100

accint0 = cpn*face/2 * (days_accint0 / days_cpn)
accint1 = cpn*face/2 * (days_accint1 / days_cpn)

fwd = (spot + accint0) * (1+repo * days_fwd/360) - accint1

fwd_spot = pd.DataFrame({'prices':[spot, fwd], 'carry':face*np.array([cpn,repo])}, index=['spot','forward'])
fwd_spot.loc['difference'] = [spot-fwd, (accint1-accint0) - face*(repo * days_fwd/360)]
fwd_spot.style.format('{:.4f}')

,prices,carry
spot,102.0625,4.0000
forward,102.3725,4.8500
difference,-0.3100,-0.2733


# Reminder on Forwards vs Futures

Futures are similar to forwards in terms of
* pricing
* hedging
* uses

Differ in key ways. Futures...
* trade on exchanges, not OTC.
* trade a standardized contract
* have P&L accrue daily via daily settlement
* require a maintenance margin

Of these differences, a few consequences are worth highlighting.

**Credit risk**
* The exchange takes on the credit risk by intermediating both sides of the trade. And the maintenance margin means they have little credit risk. 
* However, this difference with forwards may be overstated, as recent regulation and practice has seen more forward agreements require intermediation and maintenance margins.

# Daily Settlement

**Daily settlement**
* The daily realization of P&L does not change the total P&L cashflows (relative to a forward).
* It does change the timing. A classic result (FINM 33000?) notes that this creates a pricing impact.

### Daily Settlement Reduces Bond Futures Price
The negative correlation between rates and the futures (bond) price has a negative impact on the futures price.

Long position in futures contract on bond
* loses money when rate goes up (and loss is more expensive to finance) 
* gains when rate goes down, (and opportunity to invest profit is low.) 
* Thus, the daily settlement timing is unfavorable to long futures position in bonds. 

Futures price is less than forward price.
* But this pricing effect is small for bond futures which are short-term relative to maturity of the bond.
* And the short rate (repo) and long rates are not perfectly correlated.

### Tailing the Hedge

In practice, daily settlement is notable in how it impacts **hedging**.

Let $\pnl$ denote the daily (book) PnL on the future or forward position.

If managing daily risk, note that position's (book) value changing $\pnl$ impacts futures and forwards differently.
* Futures value changes $\pnl$ immediately due to daily settlement.
* Forward value changes by present value of $\pnl$, denoted $Z(t,T)\pnl$ given that the P&L will not be realized until the forward term, $T$.

Thus, the futures contract is more responsive to daily movements and requires a smaller position to hedge than a forward-based hedge.

$$N_t^{\text{fut}} = Z(t,T)N_t^{\text{fwd}}$$

This difference, (the reduced hedge position in futures,) is known as **tailing the hedge.**

#### The Present Value Discount
For a short-term futures, (less than a year,) typical to use the repo rate for calculating the present value discount, $Z(t,T)$, rather than the treasury curve:

$$Z(t,T) = \frac{1}{1+r^{\text{repo}}(t,T)\yearsACT}$$
where $\yearsACT$ is the forward interval measured in ACT/360 years.

***

# Bond Futures

See the details for the most-traded bond futures contract: `TY`. 
* 10-year US Treasury note future
* [CME volumes](https://www.cmegroup.com/market-data/browse-data/interest-rate-volume.html?redirect=/market-data/volume-open-interest/interest-rate-volume.html)

[CBOT contract details](https://www.cmegroup.com/markets/interest-rates/us-treasury/10-year-us-treasury-note.contractSpecs.html)

[Calendar for the future chain](https://www.cmegroup.com/markets/interest-rates/us-treasury/10-year-us-treasury-note.calendar.html)

<img src="../refs/CME_treasury_table.png" />

Reference: https://www.cmegroup.com/trading/interest-rates/basics-of-us-treasury-futures.html

## Delivery and Pricing

Bond futures embed several options (to the short) with regard to delivering the underlying bond at expiration.
* Quality
* Timing
* End-of-month
* Wildcard

We discuss these options further. For now, consider one aspect of the "quality" option.

For example, the "10"-year futures contract contains optionality with regard to which treausury can be delivered.
* Any US Treasury note with maturity between 6.5-10 years.
* One reason for this is to prevent "squeezes", microstructure issues, etc.


### Conversion factors
Of course, the deliverable bonds differ in maturity and coupon, and thus range in value.

To account for this, CBOT uses a **conversion factor** such that the futures price at delivery for bond $i$ is
$$\futpriceT\convert^i$$

These conversion factors are the value of the bond is the value using a flat YTM curve at 6\%.

Suppose bond $i$ has 
* coupon rate $\cpn_i$
* maturity $T_i$
$$\begin{align*}
\convert^{i} =& \displaystyle\sum_{j=1}^{2T_i}\frac{\cpn_i\,/\,2}{(1+.06/2)^j} + \frac{1}{(1+.06/2)^{2T_i}}\\
=& \frac{\cpn_i}{.06}\left(1-\frac{1}{(1+.06/2)^{2T_i}}\right)+\frac{1}{(1+.06/2)^{2T_i}}
\end{align*}$$ 

Note that these conversion factors are constant over the length of the futures contract. (Thus no need to subscript the conversion factor with $t$.)

### Delivery Cost

For a short position, the cost to deliver bond $i$ to close out the futures contract is
$$\text{delivery cost}_T = (P_T^i + \accint_T^i) - (\convert^i
\futpriceT + \accint_T^i) = P_T^i - \convert^i\futpriceT$$
where
* $\accint$ denotes accrued interest.
* $\psi$ denotes the conversion factor for the bond.

Note that accrued interest is received by the short delivering to close out the future.
* Thus, accrued interest cancels and is not relevant for the delivery cost.

### Cheapest to Deliver

Of course, shorts are interested in minimizing delivery cost by selecting the best deliverable bond.

Let $P_T^{\ctd}$ denote the price of the **cheapest to deliver** bond.

At expiration, no-arbitrage arguments require that the (converted) futures price converges at delivery:
$$P_T^{\ctd} = \convert^{\ctd}\futpriceT$$

Thus,
$$\futpriceT = \frac{P_T^{\ctd}}{\convert^{\ctd}} \le \frac{P_T^{i}}{\convert^{i}}, \forall i$$

<img src="../refs/treasury_futures_CTD.png" />

https://www.cmegroup.com/education/files/understanding-treasury-futures.pdf

### Understanding CTD

In a low-rate environment, (as much of 2008-2021,) which bonds tend to be CTD?

Consider how CTD would change if the term structure of rates...
* is flat at 6%.
* steepens
* flattens

Why is the notional discount of 6% used for the CTD calculation?
* Is this appropriate in rate environments far from 6%?
* Does this lead to arbitrage? And if the notional discount were 10%?

# Gross and Net Basis

## The Basis Trade
Being long the bond's basis means taking a position of
* long a forward bond at $T$
* sell the associated futures contract, adjusted for the conversion factor.

Two important bond pricing measures refer to this trade:
* **Gross basis** measures the basis trade where the bond is transacted spot. 
    * the trade includes carry of holding the bond.
* **Net basis** measures the basis trade transacting the bond forward, 
    * thus forgoing the carry.

Denote gross and net basis as $\grossbasis$ and $\netbasis$, respectively.
$$\begin{align*}
\underbrace{\grossbasis_{t,t+h}^i}_{\text{gross basis}}=&\;  P_t^i - \convert^i\futprice\\[5pt]
\underbrace{\netbasis_{t,t+h}^i}_{\text{net basis}} =&\; \fwdprice^i - \convert^i\futprice
\end{align*}$$
* where $\pfwd^i(t,T)$ denotes the $T$-forward price of bond $i$ at time $t$.

Thus, given that the forward drop equals the bond's carry, net basis can be seen as

$$\netbasis_{t,t+h}^i = \grossbasis_{t,t+h}^i - \carry_{t,t+h}^i$$

### Convergence
Note that according to the definitions above, 
* at expiration, $T$, gross basis and net basis are equal,
$$\grossbasis_T^{i} = \netbasis_T^{i}$$
* if a bond were guaranteed to be CTD, net basis for it would be zero
* at expiration, $T$, the CTD bond has net basis of 0
$$\grossbasis_T^{\ctd} = \netbasis_T^{\ctd} = 0$$

### Importance of Net Basis
Net basis is a widely quoted measure, partially due to the following two facts:

1. The P&L on a **basis trade** is simply the change in net basis.
$$\begin{align*}
\pnl_{t,t+h}^i =& N \left[\fwdpriceDT^i -\fwdprice^i\right] - N\convert^i\left[\futpriceDT-\futprice\right]\\[5pt]
=& N\left[\netbasis_{t+h}^i - \netbasis_t^i\right]
\end{align*}$$

2. The net basis can be seen as a measure of how close (likely) a bond is to being CTD.
* Given the convergence noted above, one could view net basis as a function of the probability of the bond being CTD.

## Two caveats

### Daily Settlement
The PnL of the basis trade is only approximate above.
* The equations for gross and net basis are correct.
* But they only approximate the pnl of the basis trade as they do not account for **daily settlement** of futures.

Still, this makes a minor difference to the PnL in most circumstances. Furthermore, in terms of the net long-short position, it can be accounted for by setting the number of futures contracts to be adjusted for both
* conversion factor, as noted above
* tailing the hedge

### Repo
The trades above rely on the synthetic forward, which considers **term** repo--that is, a repo rate locked in for a term between $t$ and $T.

The repo often quoted, (and sometimes used in this trade!) is **overnight** repo.
* In that case, there is additional basis for overnight vs term repo.
* PnL may diverge further.

## Implied Repo

Another common metric in quoting bond prices is **implied repo**. 
* Gives the return to going long spot and shorting via futures (instead of forward).
* Each bond has an implied repo, which assumes that particular bond will become CTD.
* Thus, if implied repo is higher than actual repo, and the bond is sure to be CTD, then it indicates the future is overpriced.

**Careful**
This implied repo is assuming the bond becomes CTD!
* A high / low implied repo may simply be impacted by the fact that bond is unlikely to become CTD.

### Computing Implied Repo
Solve for the repo rate that rationalizes the futres-as-forward trade.
1. Assume the forward price is the futures price adjusted by the conversion factor.
$$\fwdprice^i = \futprice^i\convert^i$$

2. Solve for repo rate such that this forward price fits the spot price:

$$\underbrace{\fwdprice^i + \accint^i_T}_{\text{dirty forward}} = \underbrace{\left[P^i_t + \accint_t^i\right]}_{\text{dirty spot}}\left[1+r^i_{\text{IR}}\yearsACT\right]$$
where
* $r^i_{\text{IR}}$ denotes the implied repo rate of bond $i$
* $\yearsACT$ is the forward interval in years (measured by ACT/360).

Thus we have $r^i_{\text{IR}}$ as the rate which solves
$$\underbrace{\futprice^i\convert^i + \accint^i_T}_{\text{dirty future-as-forward}} = \underbrace{\left[P^i_t + \accint_t^i\right]}_{\text{dirty spot}}\left[1+r^i_{\text{IR}}\yearsACT\right]$$
so
$$\begin{align*}
r^i_{\text{IR}} =& \frac{1}{\yearsACT}\left[\frac{\futprice^i\convert^i + \accint^i_T}{P^i_t + \accint_t^i} - 1\right]\\[5pt]
=& \frac{1}{\yearsACT}\frac{\futprice^i\convert^i - P^i_t + \accint^i_T - \accint_t^i}{P^i_t + \accint_t^i}
\end{align*}$$

This last expression clarifies that **implied repo can be seen as the (annualized) return** of a trading strategy which
* goes long the bond in the spot market, at the dirty price
* simultaneously sells the associated futures contract for term $t\to T$.
* at $T$, delivers the long bond position.

# Bond Options

## Quality Option
The **quality option** refers to the seller's optionality with regard to the CTD.

If environment has _______ rates, then expect ________ duration bonds to be CTD.
* low / low
* high / high
* mid / mid

Calculating exactly the CTD requires using a derivatives pricing model.
* our risk-neutral models on binomial trees are well suited to this task.

Above we saw that terminal futures price is
$$\futpriceT = \frac{P_T^{\ctd}}{\convert^{\ctd}} \le \frac{P_T^{i}}{\convert^{i}}, \forall i$$

But at $t$, there is uncertainty as to which bond will be CTD at $T$. Thus, we have the futures price as the lower envelope of price-conversion ratios at $T$, and strictly below that envelope at $t<T$ due to the value of this quality option.
$$\futprice < \frac{P_t^{i}}{\convert^{i}}, \forall i$$

We return later to using binomial trees to model the CTD and the futures price.

## Timing Options

### Timing Option
The seller can deliver any day of the delivery month.
* If coupon is **higher** than repo, deliver as **late** as possible
    * Collect carry
    * Retain option value
* If coupon is **lower** than repo, delivery is **ambigious**. Early delivery would
    * eliminate negative carry
    * eliminate option value

    It is an quantitative issue whether forfeiting the option value is worth eliminating the negative carry.

### End-of-Month Option
Contract trading ends ~3 weeks into the month, but seller has another week to decide delivery.
* Futures price is locked
* Spot price continues to move
* Possible that a different bond becomes CTD than the bond which set the futures price at last trade date.

### Wildcard Option
* Trading on the exchange settles at 2pm CT yet the shorts have a few more hours to decide whether to issue a notice of delivery.
* Thus, a basis trade long the CTD and hedged with futures will consider a post-settlement rise in CTD price as a profit:
    * seller might decide to deliver if the price of the CTD rises between the time the exchange settles to $F_t$ but before delivery notice is due.
* This is like the timing option--a question of whether the moneyness outweighs the ongoing option value.